In [4]:
import cv2
import numpy as np
#from skimage.measure import compare_ssim --> deprecated
from skimage.metrics import structural_similarity as compare_ssim
from PIL import Image, ImageFont, ImageDraw

In [5]:
# 웹캠이 하나인 경우 0번째
cap = cv2.VideoCapture(0)
if cap.isOpened() == False:
    print('카메라를 오픈 할 수 없습니다.')
    
frame_wdith = int(cap.get(3))
frame_height = int(cap.get(4))

old_image = None
show_image = None

while True:
    ret, frame = cap.read()
    if ret == True:
        show_image = frame.copy()
        
        if old_image is not None:
            # 흑백으로 변환
            grayA = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            grayB = cv2.cvtColor(old_image, cv2.COLOR_BGR2GRAY)
            # 1미만의 확률
            (score, diff) = compare_ssim(grayA, grayB, full=True)
            diff = (diff * 255).astype('uint8')
            # 변화하는 영역 보여주기
            cv2.imshow('DIFF', diff)
            text = '유사도: {:0.9f}'.format(score)
        
            font = ImageFont.truetype('malgun.ttf', 17)
            text_w, text_h = font.getsize(text)
            w = show_image.shape[1]
            h = show_image.shape[0]
            X_POS = w - text_w - 10
            Y_POS = h - text_h - 10
            
            # open cv --> pillow convert
            pil_image = Image.fromarray(show_image)
            draw = ImageDraw.Draw(pil_image)
            draw.text((X_POS, Y_POS), text, (255,255,255), font=font)
            show_image = np.array(pil_image)
            
            # 유사도가 90% 이하로 떨어질 경우
            if score < 0.90:
                                #  왼쪽 위 구석, 오른쪽 아래 구석, 빨강, 두께  
                cv2.rectangle(show_image, (0, 0), (w, h), (0, 0, 255), 6)
            
        old_image = frame
        cv2.imshow('CCTV', show_image)
                   
            # C style code
        if cv2.waitKey(1) & 0xFF == ord('q'):   # q를 눌러 종료
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()